# 6.4. 多输入多输出通道
## 6.4.1. 多输入通道
每个通道执行互相关操作，然后将结果相加。

In [82]:
import sys
sys.path.append('..')

In [83]:
import torch
import helper

def corr2d_multi_in(X, K):
    # 先遍历X和K的第0个维度（通道维度），再把它们加在一起
    return sum(helper.corr2d(x, k) for x, k in zip(X, K))

In [84]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 6.4.2. 多输出通道
实现一个计算多个通道的输出的互相关函数。
torch.stack()将输入按某个维度叠放。

In [85]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [86]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [87]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 6.4.3. 1*1卷积层
1*1卷积核在高度和宽度上不具备特征提取功能，但在通道上可以做到提取<br>
使用全连接层实现1*1卷积:

In [88]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, m, n = X.shape
    c_o, _, _, _ = K.shape
    X = X.reshape((c_i, m * n))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, m, n))

In [89]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6